<a href="https://cognitiveclass.ai"><img src = "https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/Logos/organization_logo/organization_logo.png" width = 400> </a>

<h1 align=center><font size = 5>Peer Review Final Assignment</font></h1>

## Introduction


In this lab, you will build an image classifier using the VGG16 pre-trained model, and you will evaluate it and compare its performance to the model we built in the last module using the ResNet50 pre-trained model. Good luck!

## Table of Contents

<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 3>    

1. <a href="#item41">Download Data 
2. <a href="#item42">Part 1</a>
3. <a href="#item43">Part 2</a>  
4. <a href="#item44">Part 3</a>  

</font>
    
</div>

<a id="item41"></a>

## Download Data

Use the <code>wget</code> command to download the data for this assignment from here: https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/concrete_data_week4.zip

Use the following cells to download the data.

In [ ]:
# Download data
!wget https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/concrete_data_week4.zip

In [ ]:
# Unzip
!unzip concrete_data_week4.zip

After you unzip the data, you fill find the data has already been divided into a train, validation, and test sets.

<a id="item42"></a>

## Part 1

In this part, you will design a classifier using the VGG16 pre-trained model. Just like the ResNet50 model, you can import the model <code>VGG16</code> from <code>keras.applications</code>.

You will essentially build your classifier as follows:
1. Import libraries, modules, and packages you will need. Make sure to import the *preprocess_input* function from <code>keras.applications.vgg16</code>.
2. Use a batch size of 100 images for both training and validation.
3. Construct an ImageDataGenerator for the training set and another one for the validation set. VGG16 was originally trained on 224 × 224 images, so make sure to address that when defining the ImageDataGenerator instances.
4. Create a sequential model using Keras. Add VGG16 model to it and dense layer.
5. Compile the mode using the adam optimizer and the categorical_crossentropy loss function.
6. Fit the model on the augmented data using the ImageDataGenerators.

Use the following cells to create your classifier.

In [1]:
# Libraries
from keras.preprocessing.image import ImageDataGenerator

import keras
from keras.models import Sequential
from keras.layers import Dense

from keras.preprocessing import image

from keras.applications import VGG16
from keras.applications.vgg16 import preprocess_input

Using TensorFlow backend.


In [2]:
batch_size_training = 10
batch_size_validation = 10

In [3]:
# Construct an ImageDataGenerator
data_generator = ImageDataGenerator(preprocessing_function = preprocess_input,)

In [4]:
# Get the training images
train_generator = data_generator.flow_from_directory(
    'concrete_data_week4/train',
    target_size = (224, 224),
    batch_size = batch_size_training,
    class_mode = 'categorical')

Found 30001 images belonging to 2 classes.


In [5]:
# Get the validation images
validation_generator = data_generator.flow_from_directory(
    'concrete_data_week4/valid',
    target_size = (224, 224),
    batch_size = batch_size_validation,
    class_mode = 'categorical')

Found 9501 images belonging to 2 classes.


In [7]:
# Create a Sequencial model
model = Sequential()

# Add the VGG16 model; do not include the output/top layer
model.add(VGG16(
    include_top=False,
    pooling='avg',
    weights='imagenet',))

## Add a Dense layer
model.add(Dense(2, activation='softmax'))

Instructions for updating:
Colocations handled automatically by placer.


In [8]:
# Do not retrain the VGG16 model
model.layers[0].trainable = False

In [9]:
# See how many parameters need to be optimized in order to train the output layer
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 512)               14714688  
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 1026      
Total params: 14,715,714
Trainable params: 1,026
Non-trainable params: 14,714,688
_________________________________________________________________


In [10]:
# Compile the model using the Adam optimizer
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [11]:
# Define how many steps compose an epoch
steps_per_epoch_training = 50
steps_per_epoch_validation = 50

num_epochs = 2

In [12]:
# Fit the model
fit_history = model.fit_generator(
    train_generator,
    steps_per_epoch = steps_per_epoch_training,
    epochs = num_epochs,
    validation_data = validation_generator,
    validation_steps = steps_per_epoch_validation,
    verbose = 1,)

Instructions for updating:
Use tf.cast instead.
Epoch 1/2
50/50 [==============================] - 546s 11s/step - loss: 0.5081 - acc: 0.7280 - val_loss: 0.1756 - val_acc: 0.9640
Epoch 2/2
50/50 [==============================] - 743s 15s/step - loss: 0.1289 - acc: 0.9820 - val_loss: 0.1040 - val_acc: 0.9760


In [13]:
model.save('Classifier_VGG16_model.h5')

<a id="item43"></a>

## Part 2

In this part, you will evaluate your deep learning models on a test data. For this part, you will need to do the following:

1. Load your saved model that was built using the ResNet50 model. 
2. Construct an ImageDataGenerator for the test set. For this ImageDataGenerator instance, you only need to pass the directory of the test images, target size, and the **shuffle** parameter and set it to False.
3. Use the **evaluate_generator** method to evaluate your models on the test data, by passing the above ImageDataGenerator as an argument. You can learn more about **evaluate_generator** [here](https://keras.io/models/sequential/).
4. Print the performance of the classifier using the VGG16 pre-trained model.
5. Print the performance of the classifier using the ResNet pre-trained model.


Use the following cells to evaluate your models.

In [6]:
from keras.models import load_model

In [7]:
# Load the models
resnet_m = load_model('Classifier_resnet_model.h5')
vgg16_m = load_model('Classifier_VGG16_model.h5')

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.


/opt/conda/envs/Python36/lib/python3.6/site-packages/keras/engine/saving.py:327: UserWarning: Error in loading the saved optimizer state. As a result, your model is starting with a freshly initialized optimizer.
  warnings.warn('Error in loading the saved optimizer '


In [8]:
# Get the test images
test_generator = data_generator.flow_from_directory(
    'concrete_data_week4/test',
    target_size = (224, 224),
    shuffle = False)

Found 500 images belonging to 2 classes.


In [9]:
# VGG16 evaluation
scores_vgg16 = vgg16_m.evaluate_generator(test_generator,
                                          use_multiprocessing=False,
                                          steps = 10)      

In [10]:
# VGG16 Accuracy
print('VGG16 Model Accuracy: {0:0.4F}, Loss: {0:0.4F}'.format(scores_vgg16[1], scores_vgg16[0]))

VGG16 Model Accuracy: 0.9875, Loss: 0.9875


In [ ]:
# ResNet50 evaluation
scores_resnet = resnet_m.evaluate_generator(test_generator,
                                           use_multiprocessing=False,
                                           steps = 10)

In [ ]:
# ResNet50 Accuracy
print('ResNet50 Model Accuracy: {0:0.4F}, Loss: {0:0.4F}'.format(scores_resnet[1], scores_resnet[0]))

<a id="item44"></a>

## Part 3

In this model, you will predict whether the images in the test data are images of cracked concrete or not. You will do the following:

1. Use the **predict_generator** method to predict the class of the images in the test data, by passing the test data ImageDataGenerator instance defined in the previous part as an argument. You can learn more about the **predict_generator** method [here](https://keras.io/models/sequential/).
2. Report the class predictions of the first five images in the test set. You should print something list this:

<center>
    <ul style="list-style-type:none">
        <li>Positive</li>  
        <li>Negative</li> 
        <li>Positive</li>
        <li>Positive</li>
        <li>Negative</li>
    </ul>
</center>

Use the following cells to make your predictions.

In [9]:
# Function to print the list of results
def printres(yhat):
  for n in range(5):
    if yhat[n][0] > 0.5:
      print('Negative')
    else:
      print('Positive')

In [10]:
# Print the list with the predictions of VGG16 model
print('Predictions with VGG16 Model:')
printres(vgg16_m.predict_generator(test_generator, steps = 1))

Predictions with VGG16 Model:
Negative
Negative
Negative
Negative
Negative


In [ ]:
# Print the list with the predictions of ResNet50 model
print('Predictions with ResNet50 Model:')
printres(resnet_m.predict_generator(test_generator, steps = 1))

Predictions with ResNet50 Model:


### Thank you for completing this lab!

This notebook was created by Alex Aklson.

This notebook is part of a course on **Coursera** called *AI Capstone Project with Deep Learning*. If you accessed this notebook outside the course, you can take this course online by clicking [here](https://cocl.us/DL0321EN_Coursera_Week4_LAB1).

<hr>

Copyright &copy; 2020 [IBM Developer Skills Network](https://cognitiveclass.ai/?utm_source=bducopyrightlink&utm_medium=dswb&utm_campaign=bdu). This notebook and its source code are released under the terms of the [MIT License](https://bigdatauniversity.com/mit-license/).